In [1]:
import os 
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow as tf
from tensorflow import keras
import keras_nlp
from transformers import TFBertForMaskedLM, BertTokenizer
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_logical_device_configuration(
    gpu,
    [
        tf.config.LogicalDeviceConfiguration(15360//2),
        tf.config.LogicalDeviceConfiguration(15360//2)
    ]
    )

D:\anaconda\envs\tf-gpu-2.10.0-py-3.10\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\envs\tf-gpu-2.10.0-py-3.10\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\anaconda\envs\tf-gpu-2.10.0-py-3.10\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Using TensorFlow backend


D:\anaconda\envs\tf-gpu-2.10.0-py-3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logical_devices = tf.config.list_logical_devices("GPU")
print(f'共有 {len(gpus)} 个实体 GPU ，分别是 {gpus} \n共有 {len(logical_devices)} 个虚拟 GPU ，分别是 {logical_devices} ')

共有 1 个实体 GPU ，分别是 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')] 
共有 2 个虚拟 GPU ，分别是 [LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU')] 


In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [4]:
EPOCHS = 3
batch_size = 32
learning_rate = 1e-4
scaled_batch_size = batch_size * strategy.num_replicas_in_sync
scaled_learning_rate = learning_rate * strategy.num_replicas_in_sync

# `shuffle `
在 TensorFlow 中，`tf.data.Dataset` 是用于构建输入数据管道的一个模块，其中 `.shuffle(buffer_size=500)` 方法用于对数据集进行随机打乱，以提高模型训练的效果。
 

#### 工作原理

- 当你调用 `dataset.shuffle(buffer_size=500)` 时，TensorFlow 会创建一个大小为 `500` 的缓冲区（buffer）。在读取数据时，最初会将数据集中的前 `500` 个元素放入这个缓冲区中。
- 然后，`shuffle` 方法从这个缓冲区中随机选择一个元素进行输出（用于训练模型）。每次选择后，新的元素会被加入到缓冲区中，确保缓冲区的大小始终保持在 `500`。
- 这种方法通过在缓冲区中随机选择元素，并持续补充新元素，使得输出的数据顺序是随机的。
 

### 具体例子

假设你有一个包含 1000 个样本的数据集：

```
shuffled_dataset = dataset.shuffle(buffer_size=500)
```

- 在这个例子中，缓冲区的大小为 500，意味着 TensorFlow 会先将前 500 个元素（0 到 499）放入缓冲区中，然后从中随机选择一个输出。在输出第一个元素后，数据集的第 501 个元素会加入缓冲区。然后再从 501 个元素中随机选择一个输出，以此类推。
- 最终，这个过程将导致整个数据集被随机打乱，但打乱的程度取决于 `buffer_size`。

### 选择合适的 `buffer_size`

- **小数据集**: 对于小数据集，最好将 `buffer_size` 设置为数据集的大小（或者大于等于数据集的大小），以实现完全随机打乱。
- **大数据集**: 对于非常大的数据集，可能无法在内存中存放整个数据集，这时 `buffer_size` 可以设置为一个较大的数值，但不会超过内存容量。一个较大的缓冲区会提供更好的随机打乱效果，但会占用更多的内存。

在 TensorFlow 中，`shuffle(buffer_size)` 和 `batch(batch_size)` 是用于数据集预处理的重要函数，它们的参数 `buffer_size` 和 `batch_size` 可以是相等的，也可以是不同的。两者参数是否相等会影响数据处理的方式，但一般来说，它们不需要相等。

#  `batch `

### `shuffle(buffer_size)` 和 ` batch 函数(batch_size)` 参数

- **`buffer_size` (shuffle)**: 决定了用于随机打乱数据的缓冲区的大小。这个缓冲区用于在数据被加载时暂时存储数据，然后从中随机选择数据项，以打乱数据顺序。`buffer_size` 越大，打乱数据的效果越好，但也会消耗更多的内存。

- **`batch_size` (batch)**: 决定了每个批次包含的数据样本数量。`batch_size` 通常根据模型的训练需求和可用的硬件资源来设置。较小的 `batch_size` 会减少每次训练迭代的内存需求，但可能需要更多的迭代次数才能完成一次训练。较大 
### 总结

- **`shuffle(buffer_size)` 和 `batch(batch_size)` 参数不相等是正常且常见的情况**。
- **`buffer_size` 通常应大于或等于 `batch_size`**，以确保数据的随机性和模型的训练效果。如果 `buffer_size` 小于 `batch_size`，在极端情况下，可能会导致每个批次中的数据重复或部分有序，因为缓冲区中的数据可能不足以随机选择每个批次所需的样本数。
- **参数不相等不会导致错误**，但参数的选择会影响训练数据的随机性和模型的训练性能。


`cache()` 是 TensorFlow 中的 `tf.data.Dataset` API 提供的一个方法，用于将数据集的计算结果缓存到内存或本地存储中，以提高数据读取的效率，特别是当数据集需要多次遍历时。`cache()` 方法对于避免重复计算或从硬盘重新读取数据非常有用。

#  `cache`  

#### 1. **基本用法**

使用 `cache()` 可以在第一次遍历数据集时，将数据集内容缓存到内存中，以后再遍历时可以直接从内存中读取，而不需要重新计算或加载数据，减少从磁盘加载数据的开销，提高数据加载速度。这适用于数据量较小的情况

#### 2.  **缓存到本地文件**

如果数据集较大，不能完全装入内存，可以将缓存存储到本地文件中。你可以在 `cache()` 中指定一个文件路径作为缓存文件的位置：

```
# 将数据缓存到本地文件 'my_cache.tfdata'
dataset = dataset.cache('my_cache.tfdata')
```

在这个示例中，`cache('my_cache.tfdata')` 会将数据缓存到名为 `my_cache.tfdata` 的本地文件中。如果文件已经存在，`cache()` 方法会直接读取文件内容而不是重新计算。请确保有足够的存储空间。如果你需要改变数据预处理逻辑，需要删除缓存文件或使用不同的缓存文件名。

#### 4. **何时使用 `cache()`**

- **数据预处理开销大**: 如果数据集的预处理（如解析、解码、数据增强等）开销很大，使用 `cache()` 可以避免每次迭代都进行这些操作。
- **数据集较小**: 数据集足够小，可以完全放入内存。
- **数据集不会改变**: 在训练过程中数据集不会改变或非常少改变（如训练和验证集）。



# `prefetch`
`prefetch` 是 TensorFlow 中的 `tf.data.Dataset` API 提供的一个方法，用于在训练模型时对数据进行预取操作，以提高数据读取和训练的效率。通过使用 `prefetch`，可以在处理当前批次数据时，预先准备好下一个批次的数据，这样可以减少数据输入与模型训练之间的等待时间，从而更好地利用计算资源（如 GPU 或 CPU）。

### 1. **基本用法**

```
# 使用 prefetch 来进行预取操作，缓冲区大小为 1
dataset = dataset.prefetch(buffer_size=1)
 
```

### 2. **参数解释**

- **`buffer_size`**: 决定了在模型训练时，能够同时预取多少个元素。  
  - **值为 1**: 意味着数据集会始终保持一个元素的缓冲区（即，预取下一个批次的数据）。
  - **更大的值**: 增加了预取的数量，可能会进一步提高性能，特别是当数据预处理或加载时间相对较长时。
  - **`tf.data.AUTOTUNE`**: 自动选择缓冲区大小以最大化吞吐量和性能，TensorFlow 将根据训练的实际情况自动调整缓冲区的大小。建议在不确定具体 `buffer_size` 大小的情况下使用。

### 3. **自动调整缓存参数 (`buffer_size=tf.data.AUTOTUNE`)**

使用 `tf.data.AUTOTUNE` 是推荐的做法，因为它可以自动调整预取大小以最大化性能：

```
# 使用 prefetch，自动选择缓冲区大小
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
 
```

在这个例子中，`tf.data.AUTOTUNE` 会根据系统的硬件和数据处理负载来自动调整 `buffer_size`，从而达到最佳性能。

### `prefetch` 的使用场景

- **训练数据的吞吐量高**: 数据读取和预处理的开销较大，使用 `prefetch` 可以提高数据吞吐量。
- **并行计算**: 在 CPU 上进行数据预处理，同时在 GPU 上进行模型训练。
- **硬件加速**: 充分利用多核 CPU 和 GPU 的并行处理能力，提高整体的训练速度。


In [15]:
keras.utils.get_file(origin="https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip", extract=True,)
wiki_dir = os.path.expanduser("~/.keras/datasets/wikitext-2/")
wiki_train_ds = tf.data.TextLineDataset(wiki_dir + "wiki.train.tokens").filter(lambda x:tf.strings.length(x)>100).shuffle(buffer_size=500).batch(scaled_batch_size).cache().prefetch(tf.data.AUTOTUNE)
wiki_val_ds   = tf.data.TextLineDataset(wiki_dir + "wiki.valid.tokens").filter(lambda x:tf.strings.length(x)>100).shuffle(buffer_size=500).batch(scaled_batch_size).cache().prefetch(tf.data.AUTOTUNE)
wiki_test_ds  = tf.data.TextLineDataset(wiki_dir + "wiki.test.tokens").filter(lambda x:tf.strings.length(x)>100).shuffle(buffer_size=500).batch(scaled_batch_size).cache() .prefetch(tf.data.AUTOTUNE)

In [16]:
for b in wiki_train_ds.take(1):
    print(b[:3])    #  scaled_batch_size 个

tf.Tensor(
[b' Fey was an extra in a 1998 episode , and after watching herself , decided to diet and lost 30 pounds . She told The New York Times , " I was a completely normal weight , but I was here in New York City , I had money and I couldn \'t buy any clothes . After I lost weight , there was interest in putting me on camera . " In 1999 , McKay stepped down as head writer , which led Michaels to approach Fey for the position . She became SNL \'s first female head writer that year . '
 b' A common comment on the novel is the graphic nature of the many acts of torture and murder which are depicted in the novel . Vargas lets the reader see the <unk> of an oppressive regime with a degree of detail not often used by his <unk> in Latin American literature , as Michael Wood suggests in the London Review of Books : " Vargas Llosa ... tells us far more about the details of day @-@ to @-@ day intrigue , and the <unk> , sadistic <unk> of torture and murder . " Walter Kirn of the New York Time

In [17]:
tokenizer = BertTokenizer.from_pretrained('D:\\bert-base-uncased')

In [18]:
def encode(example):
    encoded = tokenizer(example.numpy().decode('utf-8'), 
                        truncation=True, 
                        padding='max_length', 
                        max_length=128, 
                        return_tensors='tf')
    return encoded['input_ids'], encoded['attention_mask'], encoded['input_ids']

def tf_encode(example):
    result = tf.py_function(encode, [example], [tf.int32, tf.int32, tf.int32])
    result = tuple(tf.reshape(r, [-1]) for r in result)
    return result

wiki_train_ds = wiki_train_ds.unbatch().map(tf_encode, num_parallel_calls=tf.data.AUTOTUNE).batch(scaled_batch_size)
wiki_val_ds   = wiki_val_ds.unbatch().map(tf_encode, num_parallel_calls=tf.data.AUTOTUNE).batch(scaled_batch_size)
wiki_test_ds = wiki_test_ds.unbatch().map(tf_encode, num_parallel_calls=tf.data.AUTOTUNE).batch(scaled_batch_size)

In [19]:
for A,B,C  in wiki_train_ds.take(1):
    print(A[:2])
    print()
    print(B[:2])
    print()
    print(C[:2])
    print()

tf.Tensor(
[[  101  1996  9552  1999  3418  6811  4535  2040  2453  2022 12599  2004
  17091  2024  3697  2000  4175  1012  6811  6981  2862  1996  3415  1997
   2116 17091  3005  3267  2003  4242  1998  2191 13727  1010 14958  7604
   2000  2060  5932  2040  2024  2025  2130  2315  1012  1996  5279  8662
   2508  1052  1012  5297 10035  2008  2062  2084  1015  1030  1010  1030
   4278 17091  2024  2315  1999  6811  6981  1010  6168  2010 11729  3017
   1026  4895  2243  1028  2758  2045  2024  1000  5190  2588  5190  1000
   1997  5932  1012   102     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [  101  1026  4895  2243  1028  2590 17091  2435  4125  2000  2334 24491
   2015  1010  2029  2823  9063  1996  6459  1997  3080  3164  5932  1012
   7570  7946  2018  2116  3596  5079  2000  3327 

In [20]:
with strategy.scope():
    model = TFBertForMaskedLM.from_pretrained('D:\\bert-base-uncased')
    optimizer = tf.keras.optimizers.Adam(learning_rate=scaled_learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.fit(wiki_train_ds, epochs=EPOCHS)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at D:\bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [21]:
# 19.7-3.8G